# Module 5: Offline Batch ingestion via SageMaker Processing job using Feature Store Spark Connector

---

**Note:** Please set kernel to `Python 3 (Data Science)` and select instance to `ml.t3.medium`

Please be aware, that you need to run through the notebook [m5_nb0_partition_data](https://github.com/aws-samples/amazon-sagemaker-feature-store-end-to-end-workshop/blob/main/05-module-scalable-batch-ingestion/m5_nb0_partition_data.ipynb) in this section of the workshop, to setup the needed data. 

## Contents

1. [Setup](#Setup)
1. [Create PySpark SageMaker Processing script](#Create-PySpark-SageMaker-Processing-script)
1. [Run batch ingestion job](#Run-batch-ingestion-job)
1. [Verify processing job results](#Verify-processing-job-results)



In this example, an alteranative route through the batch ingestion via PySpark Processing containers will be explored to ingest data direclty into the Offline Store. We will use the `.ingest_data()` api instead of the `.put_record()` api. 

This notebook will display how to use the the [Sagemaker Feature Store Manager](https://pypi.org/project/sagemaker-feature-store-pyspark/). 


To achieve this, the package [Sagemaker Feature Store Pyspark](https://pypi.org/project/sagemaker-feature-store-pyspark/) is needed. If you want to use other means of Spark, please see the [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-ingestion-spark-connector-setup.html) for further guidance.


# Setup

#### Imports 

In [2]:
from sagemaker.spark.processing import PySparkProcessor
from sagemaker import get_execution_role
from random import randint
import sagemaker
import logging
import boto3
import json

In [3]:
logger = logging.getLogger('__name__')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

In [4]:
logger.info(f'Using SageMaker version: {sagemaker.__version__}')

Using SageMaker version: 2.120.0


### Essentials

In [5]:
try:
    role = get_execution_role()
except:
    # for local dev, please set your sagemaker role here
    role = 'arn:aws:iam::XXXXXXXX:role/service-role/role-name'
logger.info(f'Role = {role}')
sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name
featurestore_runtime_client = sagemaker_session.boto_session.client('sagemaker-featurestore-runtime', region_name=region)
default_bucket = sagemaker_session.default_bucket()
logger.info(f'Default bucket = {default_bucket}')
prefix = 'sagemaker-feature-store'

spark_version = '3.1'

Role = arn:aws:iam::510646607739:role/service-role/AmazonSageMaker-ExecutionRole-20220809T104176
Default bucket = sagemaker-us-east-1-510646607739


### Getting the packages needed
To use the feature store manager directly, we only need two things. 
- the right `jar` that we can get from [maven](https://mvnrepository.com/artifact/software.amazon.sagemaker.featurestore/sagemaker-feature-store-spark-sdk)
- the right python packages that we can get from [PyPi](https://pypi.org/project/sagemaker-feature-store-pyspark/). From PyPi You can download the latest package and copy the `feature-store-manager.py` and `wrapper.py` to your local file system. This has already been done for you in the folder `feature-store-pyspark`. For the packages, please look into the feature_store_pyspark folder. We will need to provide both to the pyspark processor. 


You can either get the jars for needed directly from [maven](https://mvnrepository.com/artifact/software.amazon.sagemaker.featurestore/sagemaker-feature-store-spark-sdk), or simply install the feature-store-manager via pip, which will get us the jars without the hassle of copying them. 

In [6]:
%pip install sagemaker-feature-store-pyspark-{spark_version}
%pip install pyathena

Keyring is skipped due to an exception: 'keyring.backends'

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Keyring is skipped due to an exception: 'keyring.backends'

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
!mkdir -p scripts
!mkdir -p jars

In [8]:
jar_path = !feature-store-pyspark-dependency-jars
jar_path = jar_path[0]
jar_path

'/opt/conda/lib/python3.7/site-packages/feature_store_pyspark/jars/sagemaker-feature-store-spark-sdk.jar'

Now that we have all the needed packages, the PySpark processing can be created. 
Not only can we ingest into the feature store direclty, but combine it with data cleaning and feature engineering. 

## Create PySpark Processing script
The following script
- Creates a data schema for the spark dataframe. 
- Check the data and the columns for their data type assigments. 
- Ordinally encodes a column
- Scales the data 
- Transforms the column `purchased_on` to the ML ready feature `n_days_since_last_purchase`.
- Repartitions the data 
- Ingests into the offline feature store

If you are only interested in the batch ingestion, please take a look at the `batch_ingest_to_feature_store` method and copare it to the `ingest_to_feature_store` mehtod.

In [9]:
%%writefile ./scripts/batch_ingest_sm_pyspark.py
from pyspark.ml.feature import VectorAssembler, StringIndexer, MinMaxScaler
from feature_store_manager import FeatureStoreManager
from pyspark.sql.functions import udf, datediff, to_date, lit, col,isnan, when, count
from pyspark.sql.types import IntegerType, DoubleType, StructType, StructField, StringType, FloatType
from pyspark.sql import SparkSession, DataFrame
from argparse import Namespace, ArgumentParser
from pyspark.ml.linalg import Vector
from pyspark.ml import Pipeline
from datetime import datetime
import argparse
import ast
import logging
import boto3
import time
import os


logger = logging.getLogger('__name__')
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())


def parse_args() -> None:
    parser = argparse.ArgumentParser()
    parser.add_argument('--num_processes', type=int, default=1)
    parser.add_argument('--num_workers', type=int, default=1)
    parser.add_argument('--feature_group_name', type=str)
    parser.add_argument('--feature_group_arn', type=str)
    parser.add_argument('--target_feature_store_list', type=str)
    parser.add_argument('--s3_uri_prefix', type=str)
    
    args, _ = parser.parse_known_args()
    return args

def transform_row(row) -> list:
    columns = list(row.asDict())
    record = []
    for column in columns:
        feature = {'FeatureName': column, 'ValueAsString': str(row[column])}
        record.append(feature)
    return record

def ingest_to_feature_store(args: argparse.Namespace, rows) -> None:
    feature_group_name = args.feature_group_name
    session = boto3.session.Session()
    featurestore_runtime_client = session.client(service_name='sagemaker-featurestore-runtime')
    rows = list(rows)
    logger.info(f'Ingesting {len(rows)} rows into feature group: {feature_group_name}')
    for _, row in enumerate(rows):
        record = transform_row(row)
        response = featurestore_runtime_client.put_record(FeatureGroupName=feature_group_name, Record=record)
        assert response['ResponseMetadata']['HTTPStatusCode'] == 200

def batch_ingest_to_feature_store(args: argparse.Namespace, df: DataFrame) -> None:
    feature_group_name = args.feature_group_name
    logger.info(f'Feature Group name supplied is: {feature_group_name}')
    session = boto3.session.Session()

    logger.info(f'Instantiating FeatureStoreManger!')
    feature_store_manager=FeatureStoreManager()

    logger.info(f'trying to load datatypes directly from Dataframe')

    # Load the feature definitions from input schema. The feature definitions can be used to create a feature group
    feature_definitions = feature_store_manager.load_feature_definitions_from_schema(df)
    logger.info(f'Feature definitions loaded successfully!')
    print(feature_definitions)
    feature_group_arn = args.feature_group_arn
    logger.info(f'Feature Group ARN supplied is: {feature_group_arn}')

    # If only OfflineStore is selected, the connector will batch write the data to offline store directly
    args.target_feature_store_list = ast.literal_eval(args.target_feature_store_list)
    logger.info(f'Ingesting into the following stores: {args.target_feature_store_list}')

    feature_store_manager.ingest_data(input_data_frame=df, feature_group_arn=feature_group_arn, target_stores= args.target_feature_store_list) 
    logger.info(f'Feature Ingestions successful!')


def check_data(df: DataFrame) -> None:
    # Sanity checking secktion 
    logger.info(f'First 5 rows of the dataframe for inspection: {df.show(5)}')

    df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]
    ).show()

    # checking for categorical columns
    categorical_cols = [field for (field, dataType) in df.dtypes if dataType == 'string']
    logger.info(f'Categorical columns: {categorical_cols}')

    # checking for numerical columns
    numerical_cols = [field for (field, dataType) in df.dtypes if ((dataType == 'double') | (dataType == 'int') | (dataType == 'float'))]
    logger.info(f'Numerical columns: {numerical_cols}')

    # checking for boolean columns  
    boolean_cols = [field for (field, dataType) in df.dtypes if dataType == 'boolean']
    logger.info(f'Boolean columns: {boolean_cols}')

    # checking for date columns
    date_cols = [field for (field, dataType) in df.dtypes if dataType == 'date']
    logger.info(f'Date columns: {date_cols}')


def scale_col(df: DataFrame, col_name: str) -> DataFrame:
    unlist = udf(lambda x: round(float(list(x)[0]), 2), DoubleType())
    assembler = VectorAssembler(inputCols=[col_name], outputCol=f'{col_name}_vec')
    # scale an column col_name with minmax scaler and drop the original column

    scaler = MinMaxScaler(inputCol=f'{col_name}_vec', outputCol=f'{col_name}_scaled')
    pipeline = Pipeline(stages=[assembler, scaler])
    df = pipeline.fit(df).transform(df).withColumn(f'{col_name}_scaled', unlist(f'{col_name}_scaled')) \
                                       .drop(f'{col_name}_vec')
    df = df.drop(col_name)
    df = df.withColumnRenamed(f'{col_name}_scaled', col_name)
    return df

def ordinal_encode_col(df: DataFrame, col_name: str) -> DataFrame:
    indexer = StringIndexer(inputCol=col_name, outputCol=f'{col_name}_new')
    df = indexer.fit(df).transform(df)
    df = df.drop(col_name)
    df = df.withColumnRenamed(f'{col_name}_new', col_name)
    return df


def run_spark_job():

    args = parse_args()
   
    spark = SparkSession.builder.getOrCreate()
    
    # set the legacy time parser policy to LEGACY to allow for parsing of dates in the format dd/MM/yyyy HH:mm:ss, which solves backwards compatibility issues to spark 2.4
    spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

    logger.info(f'LOGGER: Using Spark-Version:{spark.version}')

    # get the total number of cores in the Spark cluster; if developing locally, there might be no executor
    try:
        spark_context = spark.sparkContext
        total_cores = int(spark_context._conf.get('spark.executor.instances')) * int(spark_context._conf.get('spark.executor.cores'))
        logger.info(f'LOGGER: Total available cores in the Spark cluster = {total_cores}')
    except:
        total_cores = 1
        logger.info('LOGGER: Could not retrieve number of total cores. Setting total cores to 1')
    
    logger.info(f'LOGGER: Reading input file from S3. S3 uri is {args.s3_uri_prefix}')

    # define the schema of the input data
    csvSchema = StructType([
        StructField("order_id", StringType(), True),
        StructField("customer_id", StringType(), False),
        StructField("product_id", StringType(), False),
        StructField("purchase_amount", FloatType(), False),
        StructField("is_reordered", IntegerType(), False),
        StructField("purchased_on", StringType(), False),
        StructField("event_time", StringType(), False)])


    # read the pyspark dataframe with a schema 
    df = spark.read.option("header", "true").schema(csvSchema).csv(args.s3_uri_prefix)  
    
    # check the data of the dataframe and write findings to logs for inspection 
    check_data(df)

    # transform 1 - encode boolean to int
    df = ordinal_encode_col(df, 'is_reordered')
    df = df.withColumn('is_reordered', df['is_reordered'].cast(IntegerType()))

    # transform 2 - min max scale `purchase_amount`
    df = df.withColumn('purchase_amount', df['purchase_amount'].cast(DoubleType()))
    df = scale_col(df, 'purchase_amount')
    
    # transform 3 - derive `n_days_since_last_purchase` column using the `purchased_on` col
    current_date = datetime.today().strftime('%Y-%m-%d')
    df = df.withColumn('n_days_since_last_purchase', datediff(to_date(lit(current_date)), to_date('purchased_on', 'yyyy-MM-dd')))
    df = df.drop('purchased_on')
    df = scale_col(df, 'n_days_since_last_purchase')
    
    
    logger.info(f'LOGGER: Number of partitions = {df.rdd.getNumPartitions()}')
    # Rule of thumb heuristic - rely on the product of #executors by #executor.cores, and then multiply that by 3 or 4
    df = df.repartition(total_cores * 3)
    logger.info(f'Number of partitions after re-partitioning = {df.rdd.getNumPartitions()}')
    logger.info(f'Feature Store ingestion start: {datetime.now().strftime("%m/%d/%Y, %H:%M:%S")}')
    batch_ingest_to_feature_store(args, df)
    logger.info(f'Feature Store ingestion complete: {datetime.now().strftime("%m/%d/%Y, %H:%M:%S")}')

if __name__ == '__main__':
    logger.info('BATCH INGESTION - STARTED')
    run_spark_job()
    logger.info('BATCH INGESTION - COMPLETED')


Overwriting ./scripts/batch_ingest_sm_pyspark.py


Next, check that our feature group names, feature definition and our arguments are in line with our expectation.

In [10]:
%store -r orders_feature_group_name
s3_uri_prefix = f's3://{default_bucket}/{prefix}/partitions/*'
# REUSE orders feature group name from module 1
feature_group_name = orders_feature_group_name 
feature_group_name 

'fscw-orders-01-19-17-06'

In [11]:
sm_client=boto3.client('sagemaker')

feature_group_description = sm_client.describe_feature_group(FeatureGroupName=feature_group_name)
feature_group_arn = feature_group_description['FeatureGroupArn']

# please specify what target stores you want to ingest into -> PySpark does not accept list as a parameter
target_feature_store_list = "['OfflineStore']" # ['OfflineStore', 'OnlineStore'] for both

feature_group_name, feature_group_arn, target_feature_store_list, s3_uri_prefix

('fscw-orders-01-19-17-06',
 'arn:aws:sagemaker:us-east-1:510646607739:feature-group/fscw-orders-01-19-17-06',
 "['OfflineStore']",
 's3://sagemaker-us-east-1-510646607739/sagemaker-feature-store/partitions/*')

In [12]:
feature_group_description["FeatureDefinitions"]

[{'FeatureName': 'order_id', 'FeatureType': 'String'},
 {'FeatureName': 'customer_id', 'FeatureType': 'String'},
 {'FeatureName': 'product_id', 'FeatureType': 'String'},
 {'FeatureName': 'purchase_amount', 'FeatureType': 'Fractional'},
 {'FeatureName': 'is_reordered', 'FeatureType': 'Integral'},
 {'FeatureName': 'event_time', 'FeatureType': 'String'},
 {'FeatureName': 'n_days_since_last_purchase', 'FeatureType': 'Fractional'}]

# Run batch ingestion job
You can check what image URI for the PySpark Processor is the right one for your region via [aws github](https://github.com/aws/sagemaker-spark-container/releases).

If so, please specify the image URI via the `image_uri` attribute when instantiating the PySparkProcessor. 

Alternatively, you can specify the framework version that you would like to use via the `framework_version` attribute. 


In [13]:
from sagemaker.spark.processing import PySparkProcessor

pyspark_processor = PySparkProcessor(
    base_job_name="spark-preprocessor",
    # image_uri = image_uri,
    framework_version=spark_version,
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    max_runtime_in_seconds=1200,
)

pyspark_processor.run(submit_app='./scripts/batch_ingest_sm_pyspark.py', 
                      arguments = ['--feature_group_name', feature_group_name, 
                                   '--s3_uri_prefix', s3_uri_prefix,
                                   '--feature_group_arn', feature_group_arn,
                                   '--target_feature_store_list', target_feature_store_list],
                      submit_jars=[jar_path],
                    submit_py_files=[
                            './feature_store_pyspark/feature_store_manager.py',
                            './feature_store_pyspark/wrapper.py'
                        ],
                      spark_event_logs_s3_uri=f's3://{default_bucket}/spark-logs', 
                      logs=False,
                      wait=True)  # set logs=True to disable logging


Job Name:  spark-preprocessor-2023-02-13-09-56-53-554
Inputs:  [{'InputName': 'py-files', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-510646607739/spark-preprocessor-2023-02-13-09-56-53-554/input/py-files', 'LocalPath': '/opt/ml/processing/input/py-files', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'jars', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-510646607739/spark-preprocessor-2023-02-13-09-56-53-554/input/jars', 'LocalPath': '/opt/ml/processing/input/jars', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-510646607739/spark-preprocessor-2023-02-13-09-56-53-554/input/code/batch_ingest_sm_pyspark.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3In

# Verify processing job results

In [14]:
order_id =  f'O{randint(1, 100000)}'
logger.info(f'order_id={order_id}') 
print(feature_group_name)
feature_record = featurestore_runtime_client.get_record(FeatureGroupName=feature_group_name, 
                                                        RecordIdentifierValueAsString=order_id)
print(json.dumps(feature_record, indent=2))

order_id=O89065


fscw-orders-01-19-17-06
{
  "ResponseMetadata": {
    "RequestId": "302349b8-fbc4-494d-969c-b29c49dd92b0",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "x-amzn-requestid": "302349b8-fbc4-494d-969c-b29c49dd92b0",
      "content-type": "application/json",
      "content-length": "15",
      "date": "Mon, 13 Feb 2023 10:03:11 GMT"
    },
    "RetryAttempts": 0
  }
}


In [15]:
import pyathena as pa
import pandas as pd

# getting the latest fetaure group description
feature_group_description = sm_client.describe_feature_group(FeatureGroupName=feature_group_name)

# Opening a connection to Athena
conn = pa.connect(s3_staging_dir=f's3://{default_bucket}/athena-staging',
                    region_name=region)

# Getting the table name from the feature group description
table_name = feature_group_description['OfflineStoreConfig']['DataCatalogConfig']['TableName']

# Querying the table
query = f"""SELECT * FROM \"sagemaker_featurestore\".\"{table_name}\" 
        ORDER BY "write_time" DESC
        LIMIT 1000;"""

df = pd.read_sql(query, conn)
df

,order_id,customer_id,product_id,purchase_amount,is_reordered,event_time,n_days_since_last_purchase,write_time,api_invocation_time,is_deleted
0,O13572,C2674,P2514,0.59,0,2023-01-19T17:05:37.531Z,0.31,2023-02-13 10:02:27.432,2023-02-13 10:02:27.432,False
1,O4350,C940,P6345,0.97,0,2023-01-19T17:05:36.888Z,0.60,2023-02-13 10:02:27.432,2023-02-13 10:02:27.432,False
2,O44151,C5288,P14795,0.89,0,2023-01-19T17:05:39.759Z,0.35,2023-02-13 10:02:27.432,2023-02-13 10:02:27.432,False
3,O92041,C4586,P12821,0.90,0,2023-01-19T17:05:43.175Z,0.60,2023-02-13 10:02:27.432,2023-02-13 10:02:27.432,False
4,O52496,C7786,P1289,0.69,0,2023-01-19T17:05:40.345Z,0.60,2023-02-13 10:02:27.432,2023-02-13 10:02:27.432,False
...,...,...,...,...,...,...,...,...,...,...
995,O74165,C6823,P16144,0.57,0,2023-01-19T17:05:41.864Z,0.43,2023-02-13 10:02:27.432,2023-02-13 10:02:27.432,False
996,O38351,C5200,P3823,0.44,0,2023-01-19T17:05:39.337Z,0.62,2023-02-13 10:02:27.432,2023-02-13 10:02:27.432,False
997,O18547,C7394,P12260,0.53,0,2023-01-19T17:05:37.876Z,0.45,2023-02-13 10:02:27.432,2023-02-13 10:02:27.432,False
998,O79365,C1507,P4729,0.26,0,2023-01-19T17:05:42.225Z,0.32,2023-02-13 10:02:27.432,2023-02-13 10:02:27.432,False
